In [46]:
import praw
import csv
import os

In [47]:
user_agent = "Scraper 1.0"
reddit = praw.Reddit(
    client_id = "",
    client_secret = "",
    user_agent = user_agent
)

In [59]:
subreddit = reddit.subreddit("stocks")


In [61]:
last_post_id_file = "last_post_id.txt"

In [60]:
processed_post_ids = set()

In [62]:
if os.path.exists(last_post_id_file):
    with open(last_post_id_file, 'r') as file:
        last_processed_post_id = file.read().strip()
        if last_processed_post_id:
            processed_post_ids.add(last_processed_post_id)

In [ ]:
with open('reddit_stocks.csv', 'a', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Title', "Score", "URL", "Comments"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    if os.path.getsize('reddit_stocks.csv') == 0:
        writer.writeheader()

    for post in subreddit.top(limit=50):
        if post.id not in processed_post_ids:
            post.comments.replace_more(limit=None)
            comments = []
            for comment in post.comments.list():
                comments.append(comment.body)

            writer.writerow({
                "Title": post.title,
                "Score": post.score,
                "URL": post.url,
                "Comments": "\n".join(comments)
            })

            processed_post_ids.add(post.id)

In [53]:
if processed_post_ids:
    last_processed_post_id = max(processed_post_ids)
    with open(last_post_id_file, 'w') as file:
        file.write(last_processed_post_id)